# 4.1.2 Textlicher Teil Bebauungsplan – Embeddings + Vector Store (GPT-4o)

Das Context-Window (128.000 Tokens) in GPT-4o ist begrenzt. Entsprechend muss ab einer gewissen Seitenanzahl des schriftlichen Dokuments eines Bebauungsplans auf Emebddings zurückgegriffen werden. (~765 Tokens pro Bild mit hoher Auflösung / 85 Tokens pro Bild mit niedriger Auflösung)

* Zero-Shot
* Chain-Of-Thought

In [1]:
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from dotmap import DotMap
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from utils.parser import Parser
from utils.openai import OpenAI
from utils.runner import Runner
from utils.pprint import pprint

instructions =  "Du bist ein Assistent um Textinformationen aus dem schriftlichen Teil eines Bebauungsplans zu extrahieren."
ava = OpenAI(instructions)
parser = Parser()
runner = Runner()

load_dotenv(find_dotenv())
embeddings_model = OpenAIEmbeddings(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="text-embedding-3-large",
)

In [ ]:
# A) OCR via RapidOCR --> CNN/RNN != Transformer (LLM)
# from langchain_community.document_loaders import PyMuPDFLoader
# pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
# loader = PyMuPDFLoader(pdf_path, extract_images=True)
# data = loader.load()
# print(data[0])

In [14]:
# B1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten, um den Fokus zu maximieren und Context-Window klein zu halten.
async def page2text(page_prompt):
    msg = ava.request([
        {
            "type": "text",
            "text": 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
        },
        page_prompt
    ])
    return msg

pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F11-01-TT.pdf"
prompts = parser.pdf2prompts(pdf_path)
prompt_chain = list(map(lambda prompt: page2text(prompt), prompts))
data = await asyncio.gather(*prompt_chain)
pprint(data)

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\begin{center}
\includegraphics[width=0.2\textwidth]{logo.png} \\
19.06.2000 \\
9833
\end{center}

\begin{center}
\textbf{SCHRIFTLICHER TEIL (Teil B)} \\
\textbf{BEBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"} \\
\textbf{STADT LAICHINGEN, GEMARKUNG FELDSTETTEN, ALB-DONAU-KREIS}
\end{center}

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:

\begin{itemize}
    \item \textbf{Baugesetzbuch (BauGB)} \\
    in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    
    \item \textbf{Baunutzungsverordnung (BauNVO)} \\
    in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    
    \item \textbf{Planzeichenverordnung 1990 (PlanZV 90)} \\
    in der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 

In [2]:
# B2) OCR via GPT-4o / Kompletter schriflitcher Teil
# Idee: Komplettes PDF auf einmal einlesen, sodass Layout konsistent bleibt.
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F11-01-TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)

data2 = ava.request([
    *pdf_prompts,
    {
        "type": "text",
        "text": 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
    },
])

pprint(data2)

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\section*{Schriftlicher Teil (Teil B)}
\subsection*{Bebauungsplan "Gewerbegebiet Himmelreich"}
\subsubsection*{Stadt Laichingen, Gemarkung Feldstetten, Alb-Donau-Kreis}

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:
\begin{itemize}
    \item Baugesetzbuch (BauGB) in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    \item Baunutzungsverordnung (BauNVO) in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    \item Planzeichenverordnung 1990 (PlanzV 90) in der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 58).
    \item Landesbauordnung (LBO) in der Fassung der Bekanntmachung vom 08.08.1995 (GBl. S. 617).
\end{itemize}

\subsection*{Bisherige Festsetzungen:}
Mit in Kraft treten d

In [4]:
# C1) Ein Thread
# Nachteil: Visueller Kontext geht verloren
text_prompts = parser.text2prompts(data)
msg = ava.request([
    *text_prompts,
        {
        "type": "text",
        "text": 'Fasse relevante Informationen zu den folgenden Themen zusammen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche, Dach. Gebe deine Antwort im JSON-Format aus: {<Thema>: <Information>}. Steht keine relevante Information zur Verfügung gebe "-" aus.'
    }
])

pprint(msg)

```json
{
    "Art der baulichen Nutzung": {
        "Eingeschränktes Gewerbegebiet (GEE)": {
            "Zulässig": [
                "Gewerbebetriebe, die das Wohnen nicht wesentlich stören",
                "Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche"
            ],
            "Nicht zulässig": [
                "Nutzungen gemäß § 8 (3) Nr.3 BauNVO (Vergnügungsstätten)",
                "Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten"
            ]
        },
        "Gewerbegebiet (GE)": {
            "Zulässig": [
                "Gewerbebetriebe aller Art",
                "Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche"
            ],
            "Nicht zulässig": [
                "Nutzungen gemäß § 8 (3) Nr.3 BauNVO (Vergnügungsstätten)",
                "Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sort

In [5]:
# C2) Mehrere Threads – Extrahierter Text
# Nachteile: 
# * Visueller Kontext geht verloren
# * Fehlender Kontext zwischen den Chunks
# * Lösung enthält exkterne Logik via Vektor-Store
async def run(page_content):
    msg = ava.request([
        {
            "type": "text",
            "text": 'Fasse relevante Informationen zu den folgenden Themen zusammen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche, Dach. Gebe deine Antwort im JSON-Format aus: {<Thema>: <Information>}. Steht keine relevante Information zur Verfügung gebe "-" aus.'
        },
        {
            "type": "text",
            "text": page_content
        },
    ])
    return msg

prompt_chain = list(map(lambda page_content: run(page_content), data))
results = await asyncio.gather(*prompt_chain)
pprint(results)

In [55]:
# C3) Vector Store
# Nachteile:
# * Visueller Kontext geht verloren.
# * Größe von k schränkte vollständige Extraktion ein.
documents = list(map(lambda item: DotMap({"page_content":item[1], "metadata": DotMap({"page": item[0]+1})}), enumerate(data)))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
splits = text_splitter.split_documents(documents)
vectorstore = Chroma()
vectorstore.reset_collection()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)
query = 'Fasse relevante Informationen zu den folgenden Themen zusammen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche, Dach. Gebe deine Antwort im JSON-Format aus: {<Thema>: <Information>}. Steht keine relevante Information zur Verfügung gebe "-" aus.'
retrieved_docs = vectorstore.similarity_search_with_relevance_scores(query, k=25)
retrieved_text = list(map(lambda item: item[0].page_content, retrieved_docs))
text_prompts = parser.text2prompts(retrieved_text)

msg = ava.request([
    *text_prompts,
        {
        "type": "text",
        "text": query
    }
])

pprint(len(retrieved_docs))
pprint(msg)

25
```json
{
    "Art der baulichen Nutzung": {
        "Innenstadtrelevante Sortimente": [
            "Nahrungs- und Genussmittel einschließlich der Betriebe des Ernährungshandwerks",
            "Drogeriewaren, Parfümeriewaren",
            "Apothekenwaren",
            "Blumen, zoologischer Bedarf",
            "Oberbekleidung, Kürschnerwaren, sonstige Textilwaren, Wolle, Kurzwaren, Sportbekleidung",
            "Schuhe, Lederwaren",
            "Haushaltswaren, Unterhaltungselektronik, optische und feinmechanische Geräte, Foto- und Schreibwaren, Bücher, Spielwaren",
            "Uhren, Schmuck, Silberwaren",
            "Musikalien, Ton- und Bildträger",
            "Heimtextilien, Bettwaren, Gardinen und Zubehör"
        ],
        "Bestimmte nicht innenstadtrelevante Sortimente": [
            "Möbel, Kücheneinrichtungen, Elektrogeräte (weiße Ware), Öfen, Herde, Elektroeinbaugeräte, Haushaltstechnik",
            "Büromöbel, Büromaschinen, Computer, Büroorganisationsmittel, Büro